In [ ]:
# default_exp utility.markdown

# utility.markdown

> Markdown processing and displaying

In [ ]:
#export
from IPython.display import display, Markdown
import os.path
from pathlib import Path
import urllib.parse

In [ ]:
#export
def embed_markdown_file(file_path, display_file_link=False):
    """Embed markdown file content into ipynb.
    """
    
    with open(file_path, 'r') as markdown_file:
        text = markdown_file.read()
        display(Markdown(text))
        markdown_file.close()

In [ ]:
embed_markdown_file('limit_of_a_function_at_a_finite_input.md')

Let $f: D \to \mathbb{R}$ be a real valued function, and assume that $f$ is defined near $a$.

We write $$\lim\limits_{x \to a} f(x) = c$$ 

**Parameters**
- ``file_path`` - Path-like
- ``display_file_link`` - bool
    - If True, then displays a link to an edit mode ipynb for 
    the markdown file at ``file_path``. This should only be 
    set to True inside an ipynb environment such as jupyter. 

In [ ]:
#export
def find_links_in_markdown_text(file_path):
    """Return ranges in the markdown text string where links occur.
    """